In [20]:
import math

import numpy as np
from scipy.sparse import coo_matrix, csr_matrix, csc_matrix, lil_matrix, dok_matrix

from tqdm import tqdm

dataset_name = 'ml_small'
# dataset_name = 'ml_25m'

In [21]:
# Load the matrix

M = np.load('{}.npy'.format(dataset_name))

In [22]:
# Shuffle and split

np.random.shuffle(M)
nnz_count = M.shape[0]
row_count = int(M[:, 0].max() + 1)
col_count = int(M[:, 1].max() + 1)

training_count = round(nnz_count * 0.8)
test_count = nnz_count - training_count

M_train = M[:training_count, :]
M_test = M[training_count:, :]

# Sort M_test
# ind = np.lexsort((M_test[:, 1], M_test[:, 0]))
# M_test = M_test[ind]

In [23]:
def pearson(x, y, x_mean=None, y_mean=None):
    
    if len(x) == 0:
        return 0
    
    x_mean = x.mean()
    y_mean = y.mean()
    
    numerator = np.sum((x - x_mean) * (y - y_mean))
    denominator = np.sqrt(np.sum((x - x_mean) ** 2)) * np.sqrt(np.sum((y - y_mean) ** 2))
    
    if denominator == 0:
        return 0
    
    return numerator / denominator

In [24]:
def pearson_sparse(x, y):
    x_mean = x.sum() / x.getnnz()
    y_mean = y.sum() / y.getnnz()
    
    x = x.astype(float)
    y = y.astype(float)
    
    ind = x.nonzero()
    x[ind] -= x_mean
    
    ind = y.nonzero()
    y[ind] -= y_mean
    
    top = x.dot(y.T)[0, 0]
    bottom = math.sqrt(x.dot(x.T)[0, 0] * y.dot(y.T)[0, 0])
    
    if bottom == 0:
        return 0
    
    return top / bottom

In [6]:
M_train = [
    [0, 0, 1.0],
    [0, 1, 4.5],
    [0, 2, 2.0],
    [0, 3, 3.5],
    [0, 5, 5.0],
    [1, 0, 0.5],
    [1, 1, 5.0],
    [1, 2, 2.0],
    [1, 4, 2.0],
    [1, 5, 5.0],
    [2, 0, 1.0],
    [2, 1, 4.0],
    [2, 3, 4.0],
    [2, 4, 1.0],
    [2, 5, 4.0],
    [3, 0, 1.0],
    [3, 1, 2.0],
    [3, 2, 3.0],
    [3, 4, 4.0],
    [3, 5, 5.0],
    [4, 0, 0.5],
    [4, 1, 0.5],
    [4, 3, 4.0],
    [4, 4, 5.0],
    [4, 5, 4.0],
    [5, 0, 1.0],
    [5, 2, 1.5],
    [5, 3, 3.5],
    [5, 4, 4.0],
    [5, 5, 5.0]
]

M_test = [
    [0, 4, 1.5],
    [1, 3, 5.0],
    [2, 2, 1.0],
    [3, 3, 5.0],
    [4, 2, 0.5],
    [5, 1, 2.0]
]

M_train = np.array(M_train)
M_test = np.array(M_test)

row_count = 6
col_count = 6
training_count = 30
test_count = 6

In [7]:
M_train = [
    [0, 0, 1],
    [0, 2, 3],
    [0, 5, 5],
    [0, 8, 5],
    [0, 10, 4],
    [1, 2, 5],
    [1, 3, 4],
    [1, 6, 4],
    [1, 9, 2],
    [1, 10, 1],
    [1, 11, 3],
    [2, 0, 2],
    [2, 1, 4],
    [2, 3, 1],
    [2, 4, 2],
    [2, 6, 3],
    [2, 8, 4],
    [2, 9, 3],
    [2, 10, 5],
    [3, 1, 2],
    [3, 2, 4],
    [3, 4, 5],
    [3, 7, 4],
    [3, 10, 2],
    [4, 2, 4],
    [4, 3, 3],
    [4, 4, 4],
    [4, 5, 2],
    [4, 10, 2],
    [4, 11, 5],
    [5, 0, 1],
    [5, 2, 3],
    [5, 4, 3],
    [5, 7, 2],
    [5, 10, 4]
]

M_test = [
    [0, 4, 2]
]

M_train = np.array(M_train)
M_test = np.array(M_test)

row_count = 6
col_count = 12
training_count = M_train.shape[0]
test_count = M_test.shape[0]

In [25]:
M_train_coo = coo_matrix((M_train[:, 2], (M_train[:, 0].astype(int), M_train[:, 1].astype(int))), 
                        shape=(row_count, col_count))

# M_train_dok = M_train_coo.todok()

M_train_csr = M_train_coo.tocsr()

In [26]:
M_test_coo = coo_matrix((M_test[:, 2], (M_test[:, 0].astype(int), M_test[:, 1].astype(int))), 
                        shape=(row_count, col_count))

M_test_dok = M_test_coo.todok()

M_test_csr = M_test_coo.tocsr()

In [27]:
o_mean = M_train[:, 2].mean()

row2mean = np.zeros(row_count)
row2count = np.zeros(row_count)

col2mean = np.zeros(col_count)
col2count = np.zeros(col_count)

for nnz in M_train:
    row, col, rating = nnz
    row = int(row)
    col = int(col)
    row2count[row] += 1
    row2mean[row] += rating
    
    col2count[col] += 1
    col2mean[col] += rating

row2mean[row2count == 0] = o_mean
col2mean[col2count == 0] = o_mean
row2count[row2count == 0] = 1
col2count[col2count == 0] = 1

row2mean /= row2count
col2mean /= col2count

row2deviation = row2mean - o_mean
col2deviation = col2mean - o_mean

In [28]:
overall_mean = M_train_csr.sum() / M_train_csr.getnnz()
col_deviation = M_train_csr.sum(axis=0).A1 / M_train_csr.getnnz(axis=0) - overall_mean
col_deviation = np.nan_to_num(col_deviation, copy=False, nan=0)
row_deviation = M_train_csr.sum(axis=1).A1 / M_train_csr.getnnz(axis=1) - overall_mean
row_deviation = np.nan_to_num(row_deviation, copy=False, nan=0)

/home/seljuk/miniconda3/envs/eee/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


In [29]:
print(np.sum(~(row2deviation == row_deviation)))
print(row_count)
print(np.sum(~(col2deviation == col_deviation)))
print(col_count)

0
610
0
9724


In [30]:
print(o_mean)
print(row2deviation)
print(col2deviation)

3.502138367898449
[ 8.52700342e-01  4.97861632e-01 -8.40848045e-01 -1.88050346e-02
  1.03922238e-01 -2.13836790e-03 -2.29411095e-01  9.78616321e-02
 -1.77814044e-01 -1.71703585e-01  3.14188163e-01  7.97861632e-01
  9.78616321e-02 -6.62409320e-02 -7.87149445e-02  2.44849584e-01
  6.99011057e-01  2.30871341e-01 -9.14005733e-01  1.17653299e-01
 -2.66601178e-01 -9.91027257e-01  1.34703737e-01  1.30840356e-01
  1.33877072e+00 -2.38980473e-01  7.96798139e-02 -4.68876565e-01
  6.22861632e-01  1.26571877e+00  4.18914264e-01  2.17861632e-01
  3.10869762e-01 -5.54717012e-02  5.97861632e-01 -8.91027257e-01
  4.31194965e-01 -2.44562610e-01  5.22552990e-01  3.31194965e-01
 -2.25667780e-01  4.75853890e-02  1.02474335e+00 -1.52138368e-01
  3.51444187e-01  6.05969740e-01 -4.57422921e-01  5.57861632e-01
  7.47861632e-01 -7.41437979e-01  2.62773913e-01  9.39825918e-01
  1.49786163e+00 -4.56683822e-01 -5.02138368e-01  2.25134359e-01
 -1.33438103e-01  4.64528299e-01  8.39633784e-01  2.62567514e-01
  4.978

In [31]:
print(overall_mean)
print(row_deviation)
print(col_deviation)

3.502138367898449
[ 8.52700342e-01  4.97861632e-01 -8.40848045e-01 -1.88050346e-02
  1.03922238e-01 -2.13836790e-03 -2.29411095e-01  9.78616321e-02
 -1.77814044e-01 -1.71703585e-01  3.14188163e-01  7.97861632e-01
  9.78616321e-02 -6.62409320e-02 -7.87149445e-02  2.44849584e-01
  6.99011057e-01  2.30871341e-01 -9.14005733e-01  1.17653299e-01
 -2.66601178e-01 -9.91027257e-01  1.34703737e-01  1.30840356e-01
  1.33877072e+00 -2.38980473e-01  7.96798139e-02 -4.68876565e-01
  6.22861632e-01  1.26571877e+00  4.18914264e-01  2.17861632e-01
  3.10869762e-01 -5.54717012e-02  5.97861632e-01 -8.91027257e-01
  4.31194965e-01 -2.44562610e-01  5.22552990e-01  3.31194965e-01
 -2.25667780e-01  4.75853890e-02  1.02474335e+00 -1.52138368e-01
  3.51444187e-01  6.05969740e-01 -4.57422921e-01  5.57861632e-01
  7.47861632e-01 -7.41437979e-01  2.62773913e-01  9.39825918e-01
  1.49786163e+00 -4.56683822e-01 -5.02138368e-01  2.25134359e-01
 -1.33438103e-01  4.64528299e-01  8.39633784e-01  2.62567514e-01
  4.978

In [32]:
K = 10

rmse = 0
test_sample_count = 0

base_rmse = 0
aver_rmse = 0
wave_rmse = 0
bave_rmse = 0

for i in tqdm(range(row_count)):
# for i in [0]:
    # temp, col_i_lst = M_train_csr[i, :].nonzero()
    
    # col_i_set = set(col_i_lst)
    sim_vect = np.zeros(row_count)
    
    row_i = M_train_csr[i, :]
    
    
    for j in range(row_count):
        
        temp, col_j_lst = M_train_csr[j, :].nonzero()
        
        # common_index_lst = np.intersect1d(col_i_lst, col_j_lst) # Alternative
        # common_index_lst = list(set(col_j_lst) & col_i_set)
        
        # i_rating_lst = M_train_csr[i, common_index_lst].toarray().flatten()
        # j_rating_lst = M_train_csr[j, common_index_lst].toarray().flatten()
        row_j = M_train_csr[j, :]
        
        # sim_vect[j] = np.corrcoef(i_rating_lst, j_rating_lst)[0, 1] # Alternative
        # sim_vect[j] = pearson(i_rating_lst, j_rating_lst)
        sim_vect[j] = pearson_sparse(row_i, row_j)
    
    # print('sim_vect for user {} : '.format(i))
    # print(sim_vect)
    
    # Go over test ratigns of user i
    test, col_lst = M_test_csr[i, :].nonzero()
    
    for col in col_lst:
        rating_true = M_test_dok[i, col]
        
        # Estimate i, col
        # print('Estimating ({}, {})..'.format(i, col))
        # print('Row deviation: {}'.format(row_deviation[i]))
        # print('Col deviation: {}'.format(col_deviation[col]))
        # print('True rating: {}'.format(rating_true))
        
        # Find K-neighbors
        row_lst, temp = M_train_csr[:, col].nonzero()
        
        # Get max K values
        # We're using this solution to find max K values in linear time: 
        # https://stackoverflow.com/a/23734295/9320666
        
        if len(row_lst) < K:
            neighbor_lst = row_lst
        else:
            ind = np.argpartition(sim_vect[row_lst], -K)[-K:]
            neighbor_lst = row_lst[ind]
        neighbor_sim_lst = sim_vect[neighbor_lst]
            
        # Estimate
        baseline_estimate = overall_mean + row_deviation[i] + col_deviation[col]
        
        # Baseline estimate
        rating_estimate = baseline_estimate
        # print(rating_estimate)
        
        base_rmse += (rating_estimate - rating_true) ** 2
        
        # Simple Average
   
        rating_estimate = 0
        for neighbor in neighbor_lst:
            rating_estimate += M_train_csr[neighbor, col]
        
        if len(neighbor_lst) > 0:
            rating_estimate /= len(neighbor_lst)
        else:
            rating_estimate = baseline_estimate
        
        # print(rating_estimate)
        aver_rmse += (rating_estimate - rating_true) ** 2

        # Weighted Average
        rating_estimate = 0
        weight_sum = 0
        for neighbor, weight in zip(neighbor_lst, neighbor_sim_lst):
            if weight <= 0:
                continue
            # rating_estimate += M_train_dok[neighbor, col] * weight
            rating_estimate += M_train_csr[neighbor, col] * weight
            weight_sum += weight
            # print(neighbor, weight)

        if weight_sum == 0:
            rating_estimate = baseline_estimate
        else:
            rating_estimate /= weight_sum
        # print(rating_estimate)
        
        wave_rmse += (rating_estimate - rating_true) ** 2
        
        # Row-Col Average Considered Weighted Average
        rating_estimate = 0
        weight_sum = 0
        
        for neighbor, weight in zip(neighbor_lst, neighbor_sim_lst):
            if weight <= 0:
                continue
            rating_estimate += (M_train_csr[neighbor, col] - 
                                (overall_mean + row_deviation[neighbor] + col_deviation[col])) * weight
            weight_sum += weight
        
        if weight_sum == 0:
            rating_estimate = baseline_estimate
        else:
            rating_estimate = baseline_estimate + (rating_estimate / weight_sum)
        # print(rating_estimate)
        
        # print('Baseline Estimate is {}'.format(baseline_estimate))
        # print('Estimate is {}'.format(rating_estimate))
        # print()
        
        bave_rmse += (rating_estimate - rating_true) ** 2
        
        # Calculate error
        # rmse += (rating_estimate - rating_true) ** 2
        test_sample_count += 1
    
# rmse = math.sqrt(rmse / test_sample_count)
# print(rmse)

100%|██████████| 610/610 [13:06<00:00,  1.29s/it]


In [33]:
base_rmse = math.sqrt(base_rmse / test_sample_count)
print(base_rmse)

aver_rmse = math.sqrt(aver_rmse / test_sample_count)
print(aver_rmse)

wave_rmse = math.sqrt(wave_rmse / test_sample_count)
print(wave_rmse)

bave_rmse = math.sqrt(bave_rmse / test_sample_count)
print(bave_rmse)

0.9096434082719521
0.9674630207556456
0.9702916678642258
0.8913173824892957


In [17]:
# np.seterr(all='raise')
np.seterr(all='warn')
# np.seterr(all='print')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}